In [ ]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:
# Import aircraft info
aircraft_info = pd.read_excel('Aircraft_fleet_info.xlsx')

# Transpose the dataframe 
aircraft_info = aircraft_info.T

# Promote the first row to column headers and drop the first row
aircraft_info.reset_index(inplace=True)
aircraft_info.columns = aircraft_info.iloc[0]
aircraft_info = aircraft_info.drop(aircraft_info.index[0])

# Split the Aircraft Type column into two columns at : and drop the original column and rename the new columns
aircraft_info[['Aircraft Type', 'Aircraft Type Name']] = aircraft_info['Aircraft Type'].str.split(':', expand=True)
aircraft_info.set_index('Aircraft Type', inplace=True)

# Make it a dictionary for each aircraft type
aircraft_info = aircraft_info.to_dict('index')

# Add kerosen flag to the dictionary
aircraft_info['Type 1']['Electric'] = 0
aircraft_info['Type 2']['Electric'] = 1
aircraft_info['Type 3']['Electric'] = 1

# Make a list of aircraft types
aircraft_types = list(aircraft_info.keys())


In [ ]:
# Import distances 
distances = pd.read_csv('Group_4Regional_Distances.csv')

# Rename the unammed column to 'Origin'
distances.rename(columns={'Unnamed: 0':'Origin'}, inplace=True)
distances.set_index('Origin', inplace=True)

# Make it a dictionary
distances = distances.to_dict()

In [ ]:
# Import airport info 
airport_info = pd.read_csv('Group_4Regional_Airport_info.csv')

hub = airport_info['ICAO Code'].iloc[0]

# Make it a dictionary using the ICAO code as the key and the rest of the info in a sub-dictionary
airport_info = airport_info.set_index('ICAO Code').T.to_dict('dict')

# Create a list of the airport ICAO codes
airport_list = list(airport_info.keys())

In [ ]:
# Import demand data
daily_demand = pd.read_csv('Group_4Regional_Demand.csv')
daily_demand.rename(columns={'Unnamed: 0':'Origin'}, inplace=True)
daily_demand.set_index('Origin', inplace=True)
daily_demand = daily_demand.to_dict('index')

In [ ]:

# Import the houly coefficients
hourly_coefficients = pd.read_csv('Group_4Hourly_coefficients.csv')
hourly_coefficients.rename(columns={'Unnamed: 0':'id', 'Unnamed: 1':'Origin Name', 'Unnamed: 2':'Origin ICAO'}, inplace=True)
hourly_coefficients.drop(columns=['id','Unnamed: 0.1','Origin Name'], inplace=True)
hourly_coefficients.set_index('Origin ICAO', inplace=True)

# Compute the hourly demand for each OD pair
hour_demand = {}
for origin in airport_list:
    for destination in airport_list:
        od_demand = daily_demand[origin][destination]
        hour_demand[origin,destination] = hourly_coefficients  * od_demand 


In [ ]:
hourly_coefficients = hourly_coefficients.to_dict('index')

In [ ]:
hourly_coefficients

In [ ]:
# Revenue

''' Create Yield matrix dict from formula in RPK using distance matrix
    Formula: Yield = 5.9 ∙ dij^(−0.76) + 0.043 '''

yield_matrix = {}
for i in airport_list:
    yield_matrix[i] = {}
    for j in airport_list:
        if i == j:
            yield_matrix[i][j] = 0
        else:
            yield_matrix[i][j] = 5.9 * (distances[i][j] ** (-0.76)) + 0.043


In [ ]:
# Costs 
# Operational costs

''' Fixed costs = CXk  '''
fixed = {}
for k in aircraft_types:
    fixed[k] = aircraft_info[k]['Fixed Operating Cost (Per Fligth Leg)  [€]']

''' Time based costs i -> j
    Formula: CTkij = cTk ∙ dij / Vk '''
time_based = {}
for k in aircraft_types:
    time_based[k] = {}
    for i in airport_list:
        time_based[k][i] = {}
        for j in airport_list:
            time_based[k][i][j] = aircraft_info[k]['Cost per Hour'] * distances[i][j] / aircraft_info[k]['Speed [km/h]']

''' Fuel costs i -> j
    Formula: CFkij = cFk ∙ dij ∙ f / 1.5'''
fuel_gal_cost = 1.42 
fuel = {}
for k in aircraft_types:
    fuel[k] = {}
    for i in airport_list:
        fuel[k][i] = {}
        for j in airport_list:
            if aircraft_info[k]['Electric'] == 1:
                fuel[k][i][j] = 0
            else:
                fuel[k][i][j] = aircraft_info[k]['Fuel Cost Parameter'] * distances[i][j] * fuel_gal_cost / 1.5

''' Energy costs i -> j
    Formula: CEkij = e ∙ Gk ∙ dij / Rk '''
energy = {}
for k in aircraft_types:
    energy[k] = {}
    for i in airport_list:
        energy[k][i] = {}
        for j in airport_list:
            if aircraft_info[k]['Electric'] == 0:
                energy[k][i][j] = 0
            else:
                energy[k][i][j] = 0.07 * aircraft_info[k]['Batteries energy (kWh)'] * distances[i][j] / aircraft_info[k]['Maximum Range [km]']

''' Total costs i -> j
    Formula: CXk + CTkij + CFkij + CEkij '''
hub_factor = 0.7
op_costs = {}
for k in aircraft_types:
    op_costs[k] = {}
    for i in airport_list:
        op_costs[k][i] = {}
        for j in airport_list:
            if i == hub or j == hub:
                op_costs[k][i][j] = (fixed[k] + time_based[k][i][j] + fuel[k][i][j]) * hub_factor + energy[k][i][j]
            else:
                op_costs[k][i][j] = fixed[k] + time_based[k][i][j] + fuel[k][i][j] + energy[k][i][j]
            if i == j:
                op_costs[k][i][j] = 0
                

In [ ]:
''' Time i -> j
    Formula: dij / Vk + TATk + 30 (landing and takeoff) + additional charging time (if electric) '''
times = {}
for k in aircraft_types:
    times[k] = {}
    for i in airport_list:
        times[k][i] = {}
        for j in airport_list:
            times[k][i][j] = {}
            if i == j:
                times[k][i][j] = 0.1
            else:
                times[k][i][j] = math.ceil((distances[i][j] / aircraft_info[k]['Speed [km/h]'] * 60
                                + aircraft_info[k]['Average TAT [min]']
                                + 30
                                + (aircraft_info[k]['Additional charging time'] if aircraft_info[k]['Electric'] == 1 else 0) if i != j else 0) * 10 / 60) / 10

In [ ]:
times

In [ ]:
# Create a list of all possible time steps
time_steps = list((np.arange(0, 24, 0.1)))

for i,t in enumerate(time_steps):
    time_steps[i] = round(t, 1)

In [ ]:
hourly_coefficients[hub]['0']

In [ ]:
incoming = {}
ac = aircraft_info['Type 1']
for i in airport_list:
    incoming[i] = {}
    for t in time_steps:
        if t + times['Type 1'][i][hub] <= 24:
            arrival = round(t + times['Type 1'][i][hub],ndigits=1)
            if t >= times['Type 1'][i][hub] or times['Type 1'][i][hub] == 0.1:
                incoming[i][t, arrival] = {}
                


In [299]:
outgoing = {}
ac = aircraft_info['Type 1']
for i in airport_list:
    outgoing[i] = {}
    for t in time_steps:
        if t + times['Type 1'][hub][i] <= 24:
            arrival = round(t + times['Type 1'][hub][i], ndigits=1)
            if arrival <= 24 - times['Type 1'][hub][i] and times['Type 1'][hub][i] != 0.1:
                outgoing[i][t, arrival] = {}
outgoing

{'LIRF': {},
 'LIPZ': {(0.0, 1.7): {'pax': 0},
  (0.1, 1.8): {'pax': 0},
  (0.2, 1.9): {'pax': 0},
  (0.3, 2.0): {'pax': 0},
  (0.4, 2.1): {'pax': 0},
  (0.5, 2.2): {'pax': 0},
  (0.6, 2.3): {'pax': 0},
  (0.7, 2.4): {'pax': 0},
  (0.8, 2.5): {'pax': 0},
  (0.9, 2.6): {'pax': 0},
  (1.0, 2.7): {'pax': 0},
  (1.1, 2.8): {'pax': 0},
  (1.2, 2.9): {'pax': 0},
  (1.3, 3.0): {'pax': 0},
  (1.4, 3.1): {'pax': 0},
  (1.5, 3.2): {'pax': 0},
  (1.6, 3.3): {'pax': 0},
  (1.7, 3.4): {'pax': 0},
  (1.8, 3.5): {'pax': 0},
  (1.9, 3.6): {'pax': 0},
  (2.0, 3.7): {'pax': 0.0},
  (2.1, 3.8): {'pax': 0.0},
  (2.2, 3.9): {'pax': 0.0},
  (2.3, 4.0): {'pax': 0.0},
  (2.4, 4.1): {'pax': 0.0},
  (2.5, 4.2): {'pax': 0.0},
  (2.6, 4.3): {'pax': 0.0},
  (2.7, 4.4): {'pax': 0.0},
  (2.8, 4.5): {'pax': 0.0},
  (2.9, 4.6): {'pax': 0.0},
  (3.0, 4.7): {'pax': 5.939362313857373},
  (3.1, 4.8): {'pax': 5.939362313857373},
  (3.2, 4.9): {'pax': 5.939362313857373},
  (3.3, 5.0): {'pax': 5.939362313857373},
  (3.4, 5.1

In [ ]:
incoming[i][t, arrival]['pax'] = daily_demand[i][hub] * (hourly_coefficients[i][str(math.floor(t))]
                                                                    + (hourly_coefficients[i][str(math.floor(t-1))] if t >= 1 else 0)
                                                                    + (hourly_coefficients[i][str(math.floor(t+1))] if t <= 22.9 else 0)
                                                                    + (hourly_coefficients[i][str(math.floor(t-2))] if t >= 2 else 0))
